In [1]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
import gsw
import tables
import pandas as pd

from netCDF4 import Dataset


In [2]:
def getRHO(T,S,nlon,nlat,D):
    P=gsw.p_from_z(-D,nlat)
    SA=gsw.SA_from_SP(S, P, nlon,nlat)
    rho=gsw.rho(SA,T,P)-1000
    return rho
def store_h5(time,X):
    DF=pd.DataFrame(data={ 'MLD': X}, index=time)
    #print(DF)
    store = pd.HDFStore(cluster_data_path+'stratification/Clusters_MLD_t02'+str(YEAR)+'.h5', mode='a')
    store['DF_'+str(cluster_id)]=DF
    store.close()

In [9]:
#now load the cluster data to find rho bottom
cluster_data_path = '/home/amh001/space_fs7/DATA/NEP36-CanOE/'
clusters_ds=[]
for ii in range(6):
    cluster_files = sorted(glob.glob(f'{cluster_data_path}/From_Andrew//processed/daily/by_year/by_cluster/n_clusters_6/cluster_{ii}*.nc')) 
    #print(cluster_files    )
    cluster_ds =xr.open_mfdataset(cluster_files, chunks={'time':439}, engine='h5netcdf')
    clusters_ds.append(cluster_ds)

In [6]:
for cluster_ds in clusters_ds:
    cluster_ds['ALK_norm'] = 32.5*cluster_ds['ALK']/cluster_ds['S']
    cluster_ds['DIC_norm'] = 32.5*cluster_ds['DIC']/cluster_ds['S']
    D=cluster_ds.depths
    T=cluster_ds.T
    S=cluster_ds.S
    nlat=cluster_ds.nav_lat
    nlon=cluster_ds.nav_lon
    cluster_ds['sigma_0']=getRHO(T,S,nlon,nlat,D)

## prepare the data by extracting MLD for the clusters by year

In [12]:
for YEAR in np.arange(1996,2020):
    # Load the raw files
    #YEAR=1996
    print('YEAR', YEAR)
    datapath = '/gpfs/fs7/dfo/hpcmc/comda/fs2_comda/amh001/TOOLS/RUN_CDF/HINDCAST/MLD_NEP_1d/'
    file = f"{datapath}/mxl_NEP36-CanOE-TKE-VAH516_"+str(YEAR)+"*.nc"
    print(file)
    # Subset the data file in the same way that the cluster data was (limited to a smaller domain)
    x_slice = slice(215,713)
    y_slice = slice(180,750)
    ss = xr.open_mfdataset(file,
                           parallel=True)
    ss = ss.isel(x=x_slice, y=y_slice)

    # Load the climatology with the cluster labels for every x, y point
    cluster_clim_ds = xr.open_dataset(f"{cluster_data_path}/From_Andrew/processed/clim/clim_with_clusters_6.nc")
    print(np.nanmax(cluster_clim_ds.cluster))
    ss = ss.set_coords(['nav_lat', 'nav_lon'])

    ss["cluster"] = cluster_clim_ds["cluster"]
    ss_clusters = ss.groupby("cluster")
    cluster_labels = list(ss_clusters.groups.keys())
    print(cluster_labels)

    #print(ds)
    # now find MLD for all of the clusters and svae by year
    for cluster_id in np.arange(6):
    #cluster_id=5
        # Option 1: Create groups to only do analysis on a given one

        ssi=ss_clusters[cluster_labels[cluster_id]]#.mean()
        MLD=ssi.somxlt02
        time=MLD.time_counter.values
        print (np.shape(MLD),np.shape(time))
        MLD_t=MLD.values.mean(axis=1)
       # print(cluster_id,np.amax(Drho))
        print('saving h5 file')
        store_h5(time,MLD_t)
        

YEAR 1996
/gpfs/fs7/dfo/hpcmc/comda/fs2_comda/amh001/TOOLS/RUN_CDF/HINDCAST/MLD_NEP_1d//mxl_NEP36-CanOE-TKE-VAH516_1996*.nc
5.0
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
(366, 9308) (366,)
saving h5 file
(366, 4005) (366,)
saving h5 file
(366, 3330) (366,)
saving h5 file
(366, 4124) (366,)
saving h5 file
(366, 7978) (366,)
saving h5 file
(366, 5006) (366,)
saving h5 file
YEAR 1997
/gpfs/fs7/dfo/hpcmc/comda/fs2_comda/amh001/TOOLS/RUN_CDF/HINDCAST/MLD_NEP_1d//mxl_NEP36-CanOE-TKE-VAH516_1997*.nc
5.0
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
(365, 9308) (365,)
saving h5 file
(365, 4005) (365,)
saving h5 file
(365, 3330) (365,)
saving h5 file
(365, 4124) (365,)
saving h5 file
(365, 7978) (365,)
saving h5 file
(365, 5006) (365,)
saving h5 file
YEAR 1998
/gpfs/fs7/dfo/hpcmc/comda/fs2_comda/amh001/TOOLS/RUN_CDF/HINDCAST/MLD_NEP_1d//mxl_NEP36-CanOE-TKE-VAH516_1998*.nc
5.0
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
(365, 9308) (365,)
saving h5 file
(365, 4005) (365,)
saving h5 file
(365, 3330) (365,)
saving h5 file
(365, 4124) (